In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import SubsetRandomSampler
import random
import math
import copy

/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [12]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 64
        self.epochs = 20
        self.best_lr_list = []
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 100
        self.save_model = False
        self.gamma = 0.1
        self.alpha_max = 0.1

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('~/data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('~/data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size
)


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 34, 5, 1)
        self.conv2 = nn.Conv2d(34, 64, 5, 1)
        self.fc1 = nn.Linear(20*20*64, 512)
        self.fc2 = nn.Linear(512, 10)
        self.drop = nn.Dropout(p=0.3)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 20*20*64)
        x = self.drop(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# model is not exactully the same as the paper since it did not mention the unit of fc


In [5]:
def load_grad(temp, model):
    for net1,net2 in zip(model.named_parameters(),temp.named_parameters()):
        net2[1].grad = net1[1].grad.clone()

In [6]:
args.best_lr_list= [1]*14
print(args.best_lr_list)
del args.best_lr_list[:]
print(args.best_lr_list)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[]


In [9]:
def best_step_size_model(args, model, device, train_loader):
    
    r = np.random.randint(920)
    sampler = SubsetRandomSampler(list(range(r*args.batch_size, (r+5)*args.batch_size)))
    
    step_size_loader = torch.utils.data.DataLoader(
    datasets.MNIST('~/data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    sampler=sampler,
    batch_size=args.batch_size,
    pin_memory = True
    )
    
    best_loss = math.inf
    best_lr = 0
    best_model = Net().to(device)
    
    
    if not args.best_lr_list:
        args.alpha_max = min((1+args.gamma) * args.alpha_max, 0.1)
    elif len(args.best_lr_list) % 10 == 0:
        args.alpha_max = max(args.best_lr_list)
        del args.best_lr_list[:]
    
    
    for i in np.linspace(0, args.alpha_max, 21):
        temp = Net().to(device)
        temp_loss = 0
        temp.load_state_dict(model.state_dict())
        #load_state_dict will not copy the grad, so you need to copy it here.
        load_grad(temp, model)
        temp_optimizer = optim.SGD(temp.parameters(), lr=i)
        temp_optimizer.step()
        #optimizer will be new every time, so if you have state in optimizer, it will need load state from the old optimzer.
        
        for (data, target) in step_size_loader:
            data,target = data.to(device), target.to(device)
            output = model(data)
            temp_loss += F.nll_loss(output, target).item()
            
        if temp_loss < best_loss:
            best_model.load_state_dict(temp.state_dict())
            best_loss = copy.deepcopy(temp_loss)
            best_lr = copy.deepcopy(i)
    
    args.best_lr_list.append(best_lr)
#     print("best learning rate:", best_lr)
#     print("best loss:", best_loss)


    return best_model

In [10]:
def train(args, device, model, train_loader, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data,target = data.to(device), target.to(device)
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        
        # Chose the best step size(learning rate)
        batch_best_model = best_step_size_model(args, model, device, train_loader)
        
        model.load_state_dict(batch_best_model.state_dict())
        model.zero_grad()
        #remember to zero_grad or the grad will accumlate and the model will explode
        
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\talpha_max: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader.dataset) ,
                100. * batch_idx * args.batch_size / len(train_loader.dataset), loss.item(), args.alpha_max))
    return model

In [11]:
def test(args, device, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader)*(args.batch_size)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / (len(test_loader.dataset))))

In [ ]:
#%%time
model = Net().to(device)
# optimizer =  optim.SGD(model.parameters(), lr=args.lr)
args.best_lr_list = []

for epoch in range(1, args.epochs + 1):
    epoch_best_model = train(args, device, model, train_loader , epoch)
    model.load_state_dict(epoch_best_model.state_dict())
    test(args, device, model, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.314098
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.194167
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.140282
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.235297
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.085695
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.220732
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.101401
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.327319
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.168996
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.275723

Test set: Average loss: 0.1287, Accuracy: 9601/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.101666
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.145251
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.093275
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.239249
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.064228
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.201880
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.093321
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.283363
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.178420
T

In [133]:
for p1, p2 in zip(model1.parameters(), model2.parameters()):
    if p1.data.ne(p2.data).sum() > 0:
        print(False)
    else: print(True)

False
False
False
False
False
False
False
False
